# Reporting Conditions and Predicted Capacities

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import matplotlib as plt
import captest as ct

First we load the NREL data used for testing and set the translation dictionary.

In [ ]:
meas = ct.load_data('../../tests/data/nrel_data.csv')

In [ ]:
meas.column_groups

In [ ]:
meas.set_regression_cols(poa='irr_poa_', w_vel='wind__', t_amb='temp__', power='')

In [ ]:
meas.filter_irr(500, 600)

Below is an exmaple of calculating reporting conditions using all defaults, which are set based on the ASTM E2939 standard.  The reporting condition method always performs calculations using the filtered dataframe stored in the df_flt attribute.

- RC_irr 60% percentile
- RC_temp mean
- RC_wind mean
- Calculates values on filtered data
- inplace

In [ ]:
meas.rep_cond()

Use all defaults, except inplace
- RC_irr 60% percentile
- RC_temp mean
- RC_wind mean
- Calculates values on filtered data
- not inplace- returns rep cond and doesn't write to RCs

In [ ]:
meas.rc

In [ ]:
meas.rc = None

In [ ]:
meas.rep_cond(inplace=False)

In [ ]:
print(meas.rc)

## Irradiance Balance Method
Setting the `irr_bal` key-word argument to `True` causes the aggregation methods specified with the `func` argument to be ignored and the reporting irradiance is calculated internally by creating an instance of the `ReportingIrradiance` class. This class is intended to implement a strict interpretation of common contract language that specifies the reporting irradiance be determined by finding the irradiance that results in a balance of points within a +/- percent range of the reporting irradiance.

In [ ]:
meas.reset_filter()

In [ ]:
meas.filter_irr(500, 900)

In [ ]:
meas.rep_cond(irr_bal=True, percent_filter=20)

The instance of `ReportingIrradiance` created is stored in the `rc_tool` attribute and additional, so it can be used to access additional documentation of the reporting irradiance calculation. For example, there is a plot method, which marks the reporting irradiance selected on a linked plots showing the count of points above and below the reporting irradiance, the percent of points below the reporting irradiance, and the total points within the specifiec +/- percent band centered on the reporting irradiance.

In [ ]:
meas.rc_tool.plot()

If you have the [Panel](https://panel.holoviz.org/) package installed you can easily obtain an interactive dashboard allowing you to adjust the reporting condition parameters. *Note this dashboard will not work unless you have a live notebook.*

In [ ]:
meas.rc_tool.dashboard()

The `save_csv` method will save a copy of the following table documenting the possible options for reporting conditions to a csv file.

In [ ]:
meas.rc_tool.poa_flt

## Reporting Conditions spanning New Year Using TMY Data

The cntg_eoy year function can be used to wrap tmy data around the beginning/end of the year to prepare for calculating reporting conditions using time periods that span the new year.

In [ ]:
pvsyst = ct.load_pvsyst('./data/pvsyst_example_HourlyRes_2.CSV')

In [ ]:
pvsyst.column_groups

In [ ]:
pvsyst.set_regression_cols(power='real_pwr__', poa='irr_poa_', t_amb='temp_amb_', w_vel='wind__')

In [ ]:
pvsyst.df_flt = ct.capdata.wrap_year_end(pvsyst.data_filtered, 
                                  pd.Timestamp(year=1989, month=12, day=15),
                                  pd.Timestamp(year=1990, month=1, day=15))

In [ ]:
pvsyst.df_flt.head(2)

In [ ]:
pvsyst.df_flt.tail(2)

In [ ]:
pvsyst.filter_irr(200, 900)

In [ ]:
pvsyst.filter_pvsyst()

In [ ]:
pvsyst.rep_cond()

## Reporting Conditions for Time Periods

The freq option can be used to calculate reporting conditions for groups of time like months or seasons.  Any standard aggregation method can be used to calculate the reporting conditions for each group using the func or the irr_bal method options.

In [ ]:
pvsyst.reset_filter()

In [ ]:
print(pvsyst.data_filtered.index[0])
print(pvsyst.data_filtered.index[-1])

In [ ]:
pvsyst.filter_irr(500, 900)

In [ ]:
pvsyst.rep_cond(freq='MS')

In [ ]:
pvsyst.rc

Using the irr_bal option to calculate reporting conditions for each month of the year results in higher reporting irradiances.

Note that the with inplace set to False the reporting conditions attribute of the CapData object, pvsyst, is not overwritten.

In [ ]:
ib_rcs = pvsyst.rep_cond(freq='MS', irr_bal=True, percent_filter=20, inplace=False)
ib_rcs

In [ ]:
(ib_rcs - pvsyst.rc)['poa']

## Predicted Capacities
The `predict_capacities` method will use reporting conditions calculated using the freq option to determine predicted and guaranteed capacities.

From the previous exercise we still have the montly reporting conditions.

In [ ]:
pvsyst.rc

In [ ]:
pvsyst.tolerance = '+/- 7'

In [ ]:
pvsyst.predict_capacities()